In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import os, re, sys
from sklearn.model_selection import train_test_split
import time
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import datasets, layers, models
import gc

In [2]:
# test if GPUs are available
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# set project root, maybe you need to firstly 
# add shortcut of CS 766 Project to drive.
project_root = './drive/MyDrive/CS 766 Project/Project Coding and Data Files'

In [5]:
# class to initialize CNNs
class OriginCNN(object):
  """docstring for OriginCNN"""
  def __init__(self):
    self.optimizer = 'adam'
    # self.optimizer = tf.keras.optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
    # self.loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    self.loss = 'categorical_crossentropy'
    # will add more properties


  def build(self):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), strides=(3,3), activation='relu', input_shape=(1000, 1000, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), strides=(3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), strides=(3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(5, activation='softmax'))

    model.summary()
    model.compile(optimizer=self.optimizer,
                loss=self.loss,
                metrics=['accuracy'])
    self.model = model


  def train(self, train_images, train_labels, epochs):
    self.history = self.model.fit(train_images, train_labels, epochs=epochs, validation_split=0.1)

  def train_on_batch(self, train_images, train_labels):
    return self.model.train_on_batch(train_images, train_labels)

  def predict(self, test_images):
    return self.model.predict(test_images)


  def evaluate(self, test_images, test_labels):
    _, test_acc = self.model.evaluate(test_images, test_labels, verbose=2)
    return test_images.shape[0], test_acc

  def save_model(self, filepath):
    self.model.save(filepath)

  def load_model(self, filepath):
    self.model = tf.keras.models.load_model(filepath)

  def extract_feature(self, images):
    extract = models.Model(self.model.inputs, self.model.layers[-3].output)
    return extract.predict(images)

In [6]:
class DR_resized(object):
  """docstring for DR_resized"""
  def __init__(self, is_training, batch_id, batch_size, train_val_split_rate):
    self.batch_size = batch_size
    self.train_val_split_rate = train_val_split_rate
    if(is_training):
      # load images
      all_images = np.load(os.path.join(project_root, "Processed_Data_Batches", "Even_Class_Distribution_Datasets", "Processed_Xtrain_Batch%d_even.npy" % batch_id))
      # all_images = np.load(os.path.join(project_root, "Processed_Data_Batches", "Stratified_Random_Sampling_Datasets", "Processed_Xtrain_Batch%d.npy" % batch_id))
      # add a dimension for channels
      all_images = np.expand_dims(all_images, axis=-1)

      # load labels
      all_labels = np.load(os.path.join(project_root, "Data Batches", "Even Class Distribution Datasets", "Ytrain", "Ytrain_Batch%d_even.npy" % batch_id), allow_pickle=True)
      # all_labels = np.load(os.path.join(project_root, "Data Batches", "Stratified Random Sampling Datasets", "Ytrain", "Ytrain_Batch%d.npy" % batch_id), allow_pickle=True)
      all_labels = np.array([item[1] for item in all_labels])
      
      # one hot
      all_labels_one_hot = np.zeros((all_labels.size, 5))
      all_labels_one_hot[np.arange(all_labels.size),all_labels] = 1

      # without one hot
      # all_labels = np.expand_dims(all_labels, axis=-1)

      # shuffle
      indices = np.arange(all_images.shape[0])
      np.random.shuffle(indices)
      all_images = all_images[indices]
      all_labels_one_hot = all_labels_one_hot[indices]

      self.train_images = all_images
      self.train_labels = all_labels_one_hot
    
    else:
        # load images
      all_images = np.load(os.path.join(project_root, "Processed_Data_Batches", "Even_Class_Distribution_Datasets", "Processed_Xest_Batch%d_even.npy") % batch_id)
      # all_images = np.load(os.path.join(project_root, "Processed_Data_Batches", "Stratified_Random_Sampling_Datasets", "Processed_Xtest_Batch%d.npy") % batch_id)
      # add a dimension for channels
      all_images = np.expand_dims(all_images, axis=-1)

      # load labels
      all_labels = np.load(os.path.join(project_root, "Data Batches", "Even Class Distribution Datasets", "Ytest", "Yest_Batch%d_even.npy" % batch_id), allow_pickle=True)
      # all_labels = np.load(os.path.join(project_root, "Data Batches", "Stratified Random Sampling Datasets", "Ytest", "Ytest_Batch%d.npy" % batch_id), allow_pickle=True)
      all_labels = np.array([item[1] for item in all_labels])

      # one hot
      all_labels_one_hot = np.zeros((all_labels.size, 5))
      all_labels_one_hot[np.arange(all_labels.size),all_labels] = 1

      # without one hot
      # all_labels = np.expand_dims(all_labels, axis=-1)
      
      self.test_images = all_images
      self.test_labels = all_labels_one_hot
  
  def clear(self):
    self.train_images = None
    self.validate_images = None
    self.test_images = None
    self.train_labels = None
    self.validate_labels = None
    self.test_labels = None


In [7]:
# initialize CNN
myModel = OriginCNN()
myModel.build()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 333, 333, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 166, 166, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 55, 55, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 32)          9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0

In [10]:
# training process
file_num = 9
epoch_num = 5
for i in range(file_num):
  # load data
  myData = DR_resized(True, i, 32, 0.1)
  print("\n\ndata batch %d loaded" % i)

  for j in range(epoch_num):
    print("Start of epoch %d" % j)


    start_time = time.time()
    # matric
    train_acc_metric = keras.metrics.SparseCategoricalAccuracy()

    # train val split
    val_num = round(myData.train_val_split_rate * myData.train_images.shape[0]);
    # train dataset
    train_dataset = tf.data.Dataset.from_tensor_slices((myData.train_images[:-1 * val_num], myData.train_labels[:-1 * val_num]))
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(myData.batch_size)
    # val dataset
    val_dataset = tf.data.Dataset.from_tensor_slices((myData.train_images[-1 * val_num:], myData.train_labels[-1 * val_num:]))
    val_dataset = val_dataset.shuffle(buffer_size=1024).batch(myData.batch_size)

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

        loss_value = myModel.train_on_batch(x_batch_train, y_batch_train);

        # Update training metric.
        # train_acc_metric.update_state(y_batch_train, logits)

        # Log every n batches.
        if step % 10 == 0:
          train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
          train_logits = myModel.predict(x_batch_train)
          # Update val metrics
          train_logits = np.expand_dims(train_logits.argmax(axis=1), axis=-1)
          y_batch_train = tf.expand_dims(tf.argmax(y_batch_train, axis=1), -1)
          train_acc_metric.update_state(train_logits, y_batch_train)

          train_acc = train_acc_metric.result()
          train_acc_metric.reset_states()

          print(
              "   Training loss (for one batch) at step %d: %.4f"
              % (step, sum(loss_value) / len(loss_value))
          )
          print(
              "   Training accuracy (for one batch) at step %d: %.4f"
              % (step, train_acc)
          )
          print("   Seen so far: %d samples\n" % ((step + 1) * 32))

    # Display metrics at the end of each epoch.
    # train_acc = train_acc_metric.result()
    # print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    # train_acc_metric.reset_states()

  # Run a validation loop at the end of each epoch.
  # matric
  val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

  for x_batch_val, y_batch_val in val_dataset:
    val_logits = myModel.predict(x_batch_val)
    # Update val metrics
    val_logits = np.expand_dims(val_logits.argmax(axis=1), axis=-1)
    y_batch_val = tf.expand_dims(tf.argmax(y_batch_val, axis=1), -1)
    val_acc_metric.update_state(val_logits, y_batch_val)

  val_acc = val_acc_metric.result()
  val_acc_metric.reset_states()
  print("Validation acc: %.4f" % (float(val_acc),))
  print("data batch %d trained, time taken: %.2fs" % (i, time.time() - start_time))
  myData.clear()
  gc.collect()



data batch 0 loaded
Start of epoch 0
tf.Tensor(47, shape=(), dtype=int8)
Start of epoch 1
tf.Tensor(38, shape=(), dtype=int8)
Start of epoch 2
tf.Tensor(-10, shape=(), dtype=int8)
Start of epoch 3
tf.Tensor(121, shape=(), dtype=int8)
Start of epoch 4
tf.Tensor(31, shape=(), dtype=int8)
[[0.2053328  0.1942116  0.20771752 0.19980875 0.19292934]
 [0.19985715 0.19507903 0.21135288 0.20384535 0.18986557]
 [0.21137752 0.1914948  0.20919135 0.19970614 0.1882302 ]
 [0.21202397 0.1897097  0.21063684 0.19846411 0.18916535]
 [0.19362317 0.19248371 0.21326412 0.20768552 0.19294353]
 [0.2156697  0.18721834 0.20412368 0.19740318 0.1955851 ]
 [0.20404688 0.19842134 0.2076878  0.19438002 0.19546399]
 [0.21734373 0.18902615 0.20741889 0.194925   0.19128619]
 [0.19247189 0.20234276 0.21118543 0.2030482  0.19095175]
 [0.2012854  0.19881679 0.2107865  0.20021011 0.18890122]
 [0.20381203 0.19898324 0.20853454 0.19649221 0.19217795]
 [0.20880662 0.18319023 0.21649402 0.19940266 0.19210654]
 [0.19498774 0.

KeyboardInterrupt: ignored

In [ ]:
# testing process
file_num = 2
num_list = []
acc_list = []
for i in range(file_num):
  myData = DR_resized(False, i, 32, 0.1)
  num, acc = myModel.evaluate(myData.test_images, myData.test_labels)
  print(num, acc)
  num_list.append(num)
  acc_list.append(acc)
  myData.clear()
  gc.collect()

In [ ]:
# compute overall accuracy
num_list = np.array(num_list)
acc_list = np.array(acc_list)
acc_overall = (num_list * acc_list).sum() / num_list.sum()
print(acc_overall)

In [13]:
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y %H:%M:%S")
print("date and time =", dt_string)
myModel.save_model(os.path.join(project_root, "Trained Models", dt_string))

date and time = 23-03-2021 19:18:56
INFO:tensorflow:Assets written to: ./drive/MyDrive/CS 766 Project/Project Coding and Data Files/Trained Models/23-03-2021 19:18:56/assets


In [16]:
# this part is for loading parameters and extracting features
# load parameters
parameter_version = "23-03-2021 19:18:56"
if not os.path.exists(os.path.join(project_root, "features", parameter_version)):
    os.mkdir(os.path.join(project_root, "features", parameter_version))

my_model_pretrained = OriginCNN()
my_model_pretrained.build()
my_model_pretrained.load_model(os.path.join(project_root, "Trained Models", parameter_version))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 333, 333, 32)      320       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 166, 166, 32)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 55, 55, 32)        9248      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 27, 27, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 9, 9, 32)          9248      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 4, 4, 32)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 512)              

In [18]:
# load training data, extract features and save to npy files
file_num = 9
for i in range(file_num):
  myData = DR_resized(True, i, 32, 0.1)
  feature = my_model_pretrained.extract_feature(myData.train_images)
  # feature = myModel.extract_feature(myData.train_images)
  np.save(os.path.join(project_root, "features", parameter_version, "Xtrain_feature%d.npy" % i), feature)
  print("Xtrain_feature%d.npy" % i)
  print(feature)
  myData.clear()
  gc.collect()

# load testing data, extract features and save to npy files
file_num = 2
for i in range(file_num):
  myData = DR_resized(False, i, 32, 0.1)
  feature = my_model_pretrained.extract_feature(myData.test_images)
  np.save(os.path.join(project_root, "features", parameter_version, "Xtest_feature%d.npy" % i), feature)
  print("Xtest_feature%d.npy" % i)
  myData.clear()
  gc.collect()

Xtrain_feature0.npy
[[0.35830727 0.10113482 0.         ... 0.         0.         0.47119415]
 [0.35830727 0.04669811 0.         ... 0.         0.         0.200975  ]
 [0.47821516 0.11293279 0.         ... 0.         0.         0.31759244]
 ...
 [0.35830727 0.10821177 0.         ... 0.         0.         0.3621609 ]
 [0.38736483 0.         0.         ... 0.         0.         0.50423074]
 [0.35830727 0.12817381 0.         ... 0.         0.         0.16432361]]
Xtrain_feature1.npy
[[0.31001762 0.08447442 0.         ... 0.         0.         0.54034066]
 [0.45026395 0.04669811 0.         ... 0.         0.         0.62844735]
 [0.36011726 0.10609142 0.         ... 0.         0.         0.        ]
 ...
 [0.39272144 0.1103986  0.         ... 0.         0.         0.20643379]
 [0.37816614 0.12638164 0.         ... 0.         0.         0.36818033]
 [0.48419    0.46840778 0.         ... 0.         0.         1.2397174 ]]
Xtrain_feature2.npy
[[0.5399451  0.04669811 0.         ... 0.         0.